In [1]:
from keras.datasets import mnist
from keras.utils import np_utils


# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

# Embedding dimensions.
row_hidden = 128
col_hidden = 128





def load_data():
	# 載入mnist的資料 - same
	(x_train, y_train), (x_test, y_test) = mnist.load_data()
	# notice: reshape(,-1,1)
	x_Train=(x_train.reshape(x_train.shape[0],28,28,1).astype('float32'))/255
	x_Test=(x_test.reshape(x_test.shape[0],28,28,1).astype('float32'))/255
	# 將y轉換成one-hot encoding 
	y_TrainOneHot = np_utils.to_categorical(y_train, 10)
	y_TestOneHot = np_utils.to_categorical(y_test, 10)
	# 回傳處理完的資料
	return (x_Train, y_TrainOneHot), (x_Test, y_TestOneHot)
    

/home/waue/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense, TimeDistributed
from keras import initializers
from keras.models import Model
#改使用這些layers, Conv2D = CNN
from keras.layers import SimpleRNN
from keras.optimizers import RMSprop

def build_model(x):
	#建立模型 - same
	#model = Sequential()
	prediction = Dense(num_classes, activation='softmax')(encoded_columns)
	model = Model(x, prediction)
	model.summary()
	return model

In [3]:
from keras.layers import Input
from keras.layers import LSTM

(x_train,y_train),(x_test,y_test)=load_data()

# diff : 4d input
row, col, pixel = x_train.shape[1:]
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

model = build_model(x)
#開始訓練模型 - diff: optimizer="rmsprop"
#model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.compile(loss='categorical_crossentropy',\
              optimizer='rmsprop',\
              metrics=['accuracy'])

# diff: validation_data=(x_test, y_test))
model.fit(x_train, y_train,\
          batch_size=batch_size,\
          epochs=epochs,\
          verbose=1,\
          validation_data=(x_test, y_test))
#顯示訓練結果 - same
score = model.evaluate(x_train,y_train)
print ('\nTrain Acc:', score[1])
score = model.evaluate(x_test,y_test)
print ('\nTest Acc:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 28, 128)           66560     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 199,434
Trainable params: 199,434
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 308s 5ms/step - loss: 0.4755 - acc: 0.8432 - val_loss: 0.1517 - val_acc: 0.9547
Epoch 2/5
60000/60000 [==============================] - 307s 5ms/step - loss: 0.